In [1]:
%cd /workspace
import sys
import os

# Set PYTHONPATH environment variable for the kernel
robofin_path = os.path.join(os.getcwd(), 'robofin')
current_pythonpath = os.environ.get('PYTHONPATH', '')
if robofin_path not in current_pythonpath:
    os.environ['PYTHONPATH'] = f"{robofin_path}:{current_pythonpath}" if current_pythonpath else robofin_path

# Also add to sys.path for immediate effect
if robofin_path not in sys.path:
    sys.path.insert(0, robofin_path)

/workspace


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
def get_type(obj):
    if isinstance(obj, dict):
        if not obj:
            return "dict[?, ?]"
        key_types = {get_type(k) for k in obj.keys()}
        value_types = {get_type(v) for v in obj.values()}
        return f"dict[{', '.join(key_types)}, {', '.join(value_types)}]"
    elif isinstance(obj, list):
        if not obj:
            return "list[?]"
        elem_types = {get_type(elem) for elem in obj}
        return f"list[{', '.join(elem_types)}]"
    elif isinstance(obj, tuple):
        if not obj:
            return "tuple[?]"
        elem_types = [get_type(elem) for elem in obj]
        return f"tuple[{', '.join(elem_types)}]"
    elif isinstance(obj, set):
        if not obj:
            return "set[?]"
        elem_types = {get_type(elem) for elem in obj}
        return f"set[{', '.join(elem_types)}]"
    else:
        return type(obj).__name__

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import torch
from robofin.robots import Robot

from robofin.robot_constants import FrankaConstants

# Load the Robot class with the standard URDF file (that uses relative filepaths)
robot = Robot("assets/panda/panda.urdf")
# robot = Robot("assets/gp7/gp7.urdf")

## Sampler testing

In [5]:
from robofin.kinematics.numba import franka_arm_link_fk, franka_arm_visual_fk, franka_eef_visual_fk
from robofin.kinematics.torch import franka_arm_link_fk as torch_franka_arm_link_fk
from robofin.kinematics.torch import franka_arm_visual_fk as torch_franka_arm_visual_fk

fk = robot.fk(robot.neutral_config)

prismatic_joint = robot.auxiliary_joint_defaults["panda_finger_joint1"]
base_pose = np.eye(4)
franka_fk = franka_arm_link_fk(robot.neutral_config, prismatic_joint, base_pose)
franka_fk_dict = {
    "panda_link0": franka_fk[0],
    "panda_link1": franka_fk[1],
    "panda_link2": franka_fk[2],
    "panda_link3": franka_fk[3],
    "panda_link4": franka_fk[4],
    "panda_link5": franka_fk[5],
    "panda_link6": franka_fk[6],
    "panda_link7": franka_fk[7],
    "panda_link8": franka_fk[8],
    "panda_hand": franka_fk[9],
    "panda_grasptarget": franka_fk[10],
    "right_gripper": franka_fk[11],
    "panda_leftfinger": franka_fk[12],
    "panda_rightfinger": franka_fk[13],
}

torch_cfg = torch.Tensor(robot.neutral_config).unsqueeze(0)
torch_fk = robot.fk_torch(torch_cfg)

torch_base_pose = torch.Tensor(base_pose)
torch_franka_fk = torch_franka_arm_link_fk(torch_cfg, prismatic_joint, torch_base_pose)
torch_franka_fk_dict = {
    "panda_link0": torch_franka_fk[:,0],
    "panda_link1": torch_franka_fk[:,1],
    "panda_link2": torch_franka_fk[:,2],
    "panda_link3": torch_franka_fk[:,3],
    "panda_link4": torch_franka_fk[:,4],
    "panda_link5": torch_franka_fk[:,5],
    "panda_link6": torch_franka_fk[:,6],
    "panda_link7": torch_franka_fk[:,7],
    "panda_link8": torch_franka_fk[:,8],
    "panda_hand": torch_franka_fk[:,9],
    "panda_grasptarget": torch_franka_fk[:,10],
    "right_gripper": torch_franka_fk[:,11],
    "panda_leftfinger": torch_franka_fk[:,12],
    "panda_rightfinger": torch_franka_fk[:,13],
}

print(torch_fk["panda_hand"].shape)
print(torch_franka_fk_dict["panda_hand"].shape)


torch.Size([1, 4, 4])
torch.Size([1, 4, 4])


In [6]:
import viz_client

urdf_path = "/workspace/assets/panda/panda_collision_spheres_gltf.urdf"
if not os.path.exists(urdf_path):
    print(f"❌ URDF not found at {urdf_path}")
    print("Please update the urdf_path variable in test_connect()")
    raise FileNotFoundError(f"URDF not found at {urdf_path}")

viz_client.connect(urdf_path)

LOADED NEWW2 VIZ_CLIENT FROM WORKSPACE
Connected to viz_server


In [7]:
# hand_pose = franka_fk_dict["panda_hand"]
hand_pose = fk["panda_hand"].squeeze()

In [8]:
x,y,z = hand_pose[:3, 3]
print(f"Hand position: x={x}, y={y}, z={z}")
q_wxyz = viz_client._quaternion_trace_method(franka_fk_dict["panda_hand"])
qw,qx,qy,qz = q_wxyz
print(f"Quaternion (w, x, y, z): {qw}, {qx}, {qy}, {qz}")
ee_pose = np.concatenate((hand_pose[:3, 3], [qx, qy, qz, qw])).tolist()
viz_client.publish_ghost_end_effector(pose=ee_pose)

Hand position: x=0.31057539389779065, y=0.005652451500939399, z=0.5865411947628415
Quaternion (w, x, y, z): -0.00846602325576766, 0.9996573431250272, 0.014241942612246414, -0.020265484612719032


In [9]:
viz_client.publish_ghost_end_effector(pose=hand_pose, color=[1, 0.5, 0.2])

In [10]:
from robofin.samplers import NumpyRobotSampler, TorchRobotSampler
from robofin.samplers_original import NumpyFrankaSampler, TorchFrankaSampler

np_sampler = NumpyRobotSampler(robot)
torch_sampler = TorchRobotSampler(robot)

np_franka_sampler = NumpyFrankaSampler()
torch_franka_sampler = TorchFrankaSampler()

NOTE: Temporarily using hardcoded Franka point cloud cache
NOTE: Temporarily using hardcoded Franka point cloud cache


In [12]:
franka_ee_pts = np_franka_sampler.sample_end_effector(fk[robot.tcp_link_name].squeeze(), prismatic_joint)
ee_pts = np_sampler.sample_end_effector(fk[robot.tcp_link_name].squeeze())

print("---" * 10)
print(f"Franka EE points shape: {franka_ee_pts.shape}")
print(f"EE points shape: {ee_pts.shape}")

torch_franka_ee_pts = torch_franka_sampler.sample_end_effector(torch_franka_fk_dict[robot.tcp_link_name], prismatic_joint)
torch_ee_pts = torch_sampler.sample_end_effector(torch_fk[robot.tcp_link_name])

print("---" * 10)
print(f"Torch Franka EE points shape: {torch_franka_ee_pts.shape}")
print(f"Torch EE points shape: {torch_ee_pts.shape}")

------------------------------
Franka EE points shape: (128, 4)
EE points shape: (128, 4)
------------------------------
Torch Franka EE points shape: torch.Size([1, 128, 4])
Torch EE points shape: torch.Size([1, 128, 4])


In [ ]:
eef_link_names = ["eef_" + link for link in robot.eef_visual_link_names]
for link in eef_link_names:
    # if link in robot.eef_visual_link_names:
    print(f"Link: {link}, Points shape: {np_sampler.points[link].shape}")
    print(f"Link: {link}, Points shape: {np_franka_sampler.points[link].shape}")
    print("---" * 10)

## Compute spheres testing

In [ ]:
from robofin.samplers_original import TorchFrankaCollisionSampler

device = robot.device
c_sampler = TorchFrankaCollisionSampler(device)

batch_dim = 5
torch_neutral_config = torch.tensor(robot.neutral_config, dtype=torch.float32, device=device)
batched_configs = torch_neutral_config.unsqueeze(0).repeat(batch_dim, 1).to(device)
prismatic_joint = 0.04
spheres = c_sampler.compute_spheres(batched_configs, prismatic_joint=prismatic_joint)


In [ ]:
print(get_type(FrankaConstants.SPHERES))
print(get_type(c_sampler.spheres))
print(get_type(robot.spheres))
print(get_type(spheres))
print(get_type(robot.compute_spheres(batched_configs)))

## FK testing

In [ ]:
from robofin.kinematics.numba import franka_eef_link_fk

prismatic_joint = robot.auxiliary_joint_defaults["panda_finger_joint1"]
link8_pose = np.eye(4)
franka_fk = franka_eef_link_fk(prismatic_joint, base_pose=link8_pose)  # always relative to link8
robot_fk = robot.eef_fk(pose=link8_pose, frame="panda_link8")

franka_fk_dict = {
    "panda_link8": franka_fk[0],
    "panda_hand": franka_fk[1],
    "panda_grasptarget": franka_fk[2],
    "right_gripper": franka_fk[3],
    "panda_leftfinger": franka_fk[4],
    "panda_rightfinger": franka_fk[5],
}

print(f"prismatic_joint = {prismatic_joint}")

for link_name in franka_fk_dict:
    if not np.allclose(franka_fk_dict[link_name], robot_fk[link_name]):
        print("-"*100)
        print(f"Franka {link_name} pose:\n{franka_fk_dict[link_name]}")
        print(f"Robot {link_name} pose:\n{robot_fk[link_name]}")
        print(f"Franka {link_name} pose does not match Robot {link_name} pose")
        
    

In [ ]:
from robofin.robot_constants import FrankaConstants

to_link = "panda_link8"

print(f"Franka fixed transform {robot.tcp_link_name} -> {to_link}: \n", FrankaConstants.EEF_T_LIST[(to_link, robot.tcp_link_name)].inverse.matrix)
print(f"Robot fixed transform {robot.tcp_link_name} -> {to_link}: \n", robot.fixed_eef_link_transforms[to_link])

In [ ]:
import numpy as np
import torch
from typing import Dict

np_configs: np.ndarray = np.array([robot.neutral_config, robot.neutral_config])
torch_configs: torch.Tensor = torch.tensor(np_configs)

print(np_configs.shape)
print(torch_configs.shape)

In [ ]:
fk_np: Dict[str, np.ndarray] = robot.fk(robot.neutral_config)
fk_torch: Dict[str, torch.Tensor] = robot.fk_torch(torch.tensor(robot.neutral_config))

In [ ]:
np_visual_fk = robot.visual_fk(np_configs)
torch_visual_fk = robot.visual_fk_torch(torch_configs)

In [ ]:
print(fk_np.keys())
print(fk_torch.keys())
fk_torch["right_gripper"]

print(np_visual_fk.keys())
print(torch_visual_fk.keys())

In [ ]:

np_fk_gripper = fk_np["right_gripper"]
torch_fk_gripper = fk_torch["right_gripper"].detach().cpu().numpy()

print("Numpy FK shape:", np_fk_gripper.shape)
print("Torch FK shape:", torch_fk_gripper.shape)
print("Difference (max abs):", np.abs(np_fk_gripper - torch_fk_gripper).max())

# Optionally, test visual FK as well
if hasattr(robot, "visual_fk") and hasattr(robot, "visual_fk_torch"):
    np_visual_fk = robot.visual_fk(np_configs)
    torch_visual_fk = robot.visual_fk_torch(torch_configs)
    np_vis_gripper = np_visual_fk["panda_hand"]
    torch_vis_gripper = torch_visual_fk["panda_hand"].detach().cpu().numpy()
    print("Visual FK difference (max abs):", np.abs(np_vis_gripper - torch_vis_gripper).max())

In [ ]:
np_visual_fk = robot.visual_fk(np_configs, link_name="panda_hand")
torch_visual_fk = robot.visual_fk_torch(torch_configs, link_name="panda_hand")

In [ ]:

from robofin.samplers import NumpyRobotSampler
from robofin.samplers_original import NumpyFrankaSampler

franka_visual_fk = franka_arm_visual_fk(robot.neutral_config, 0.02, np.eye(4))

sampler = NumpyRobotSampler(robot=robot, num_robot_points=1000, num_eef_points=100, use_cache=True)
sampler.sample(robot.neutral_config, 0.02, num_points=100)

## Dataloader testing

In [ ]:
from avoid_everything.data_loader import DataModule

data_module_parameters = {
    "data_dir": "/workspace/datasets/ae_aristotle1_5mm_cubbies",
    "train_trajectory_key": "global_solutions",
    "val_trajectory_key": "global_solutions",
    "num_obstacle_points": 4096,
    "random_scale": 0.015
}
shared_parameters = {
    "prismatic_joint": 0.04,
    "num_robot_points": 2048,
    "num_target_points": 128,
    "action_chunk_length": 1
}
cfg = {
    "train_batch_size": 10,
    "val_batch_size": 10,
    "num_workers": 4,
}

dm = DataModule(
    train_batch_size=cfg["train_batch_size"],
    val_batch_size=cfg["val_batch_size"],
    num_workers=cfg["num_workers"],
    **data_module_parameters,
    **shared_parameters,
)

dm.setup()

dl = dm.train_dataloader()
sample = dl.dataset[100]

In [ ]:
import numpy as np
import torch

def convert_to_numpy_f32(arr: np.ndarray | torch.Tensor) -> np.ndarray:
    """
    Convert a NumPy array or Torch tensor to a NumPy float32 array.
    
    Parameters
    ----------
    arr : np.ndarray or torch.Tensor
        Input array to convert.
    
    Returns
    -------
    np.ndarray
        Converted array with dtype float32.
    """
    if isinstance(arr, torch.Tensor):
        np_arr: np.ndarray = arr.cpu().numpy()
    elif isinstance(arr, np.ndarray):
        np_arr: np.ndarray = arr
    else:
        raise TypeError("convert_to_numpy_f32: Input must be a NumPy array or Torch tensor")
    return np_arr.astype(np.float32) 

In [ ]:
cuboid_dims = convert_to_numpy_f32(sample["cuboid_dims"])
cuboid_centers = convert_to_numpy_f32(sample["cuboid_centers"])
cuboid_quaternions = convert_to_numpy_f32(sample["cuboid_quats"])
for dims, center, quat in zip(cuboid_dims, cuboid_centers, cuboid_quaternions):
    print(f"Lengths: Dimensions: {len(dims)}, Center: {len(center)}, Quaternion: {len(quat)}")

cylinder_radii = convert_to_numpy_f32(sample["cylinder_radii"])
cylinder_heights = convert_to_numpy_f32(sample["cylinder_heights"])
cylinder_centers = convert_to_numpy_f32(sample["cylinder_centers"])
cylinder_quaternions = convert_to_numpy_f32(sample["cylinder_quats"])
for radius, height, center, quat in zip(cylinder_radii, cylinder_heights, cylinder_centers, cylinder_quaternions):
    print(f"Lengths: Radius: {len(radius)}, Height: {len(height)}, Center: {len(center)}, Quaternion: {len(quat)}")

In [ ]:
import viz_client

urdf_path = "/workspace/assets/panda/panda_collision_spheres_gltf.urdf"
if not os.path.exists(urdf_path):
    print(f"❌ URDF not found at {urdf_path}")
    print("Please update the urdf_path variable in test_connect()")
    raise FileNotFoundError(f"URDF not found at {urdf_path}")

viz_client.connect(urdf_path)

In [ ]:
viz_client.publish_obstacles(cuboid_centers=cuboid_centers,
                            cuboid_dims=cuboid_dims,
                            cuboid_quaternions=cuboid_quaternions,
                            cylinder_centers=cylinder_centers,
                            cylinder_radii=cylinder_radii,
                            cylinder_heights=cylinder_heights,
                            cylinder_quaternions=cylinder_quaternions)

In [ ]:
viz_client.clear_obstacles()

In [ ]:
viz_client.publish_joints(joints=robot.neutral_config_dict)

In [ ]:
viz_client.shutdown()